# Colab Setup

In [23]:
from google.colab import drive

drive.mount('/content/drive')

BASE = 'drive/My Drive/kaggle/MLB'
INPUT_DIR = f'{BASE}/input'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import pandas as pd
import calendar

# Config

In [25]:
class Config:
    data_dir = f'{INPUT_DIR}/mlb-data-v2'

# Util

In [26]:
def get_train_data(file_name):
    path = f'{Config.data_dir}/train/{file_name}.pickle'

    return pd.read_pickle(path)

def get_meta_data(file_name, format='pickle'):
    path = f'{Config.data_dir}/meta/{file_name}.{format}'
    if format == 'pickle':
        return pd.read_pickle(path)
    else:
        return pd.read_csv(path)

def get_agg_data(file_name):
    path = f'{Config.data_dir}/aggregate/{file_name}.pickle'

    return pd.read_pickle(path)

# Create Data

In [27]:
salaries = get_meta_data('mlbSalaries', 'csv')
salaries = salaries[['year', 'team', 'salary']]
salaries

,year,team,salary
0,2019,arizona-diamondbacks,6166666
1,2019,arizona-diamondbacks,6050000
2,2019,arizona-diamondbacks,5025000
3,2019,arizona-diamondbacks,4825000
4,2019,arizona-diamondbacks,4500000
...,...,...,...
2335,2021,washington-nationals,506055
2336,2021,washington-nationals,460050
2337,2021,washington-nationals,346571
2338,2021,washington-nationals,346571


In [28]:
salaries = salaries.groupby(['year', 'team']).sum()['salary'].reset_index()
salaries

,year,team,salary
0,2019,arizona-diamondbacks,62990590
1,2019,atlanta-braves,115629465
2,2019,baltimore-orioles,51471496
3,2019,boston-red-sox,163861035
4,2019,chicago-cubs,196000476
...,...,...,...
85,2021,st-louis-cardinals,124483224
86,2021,tampa-bay-rays,42733384
87,2021,texas-rangers,44930530
88,2021,toronto-blue-jays,104271393


In [29]:
def map_team_name(name):
    names = name.split('-')
    result = ''
    for n in names:
        if n == 'st':
            n = 'st.'
        result += f' {n.capitalize()}'

    return result[1:]

salaries['team'] = salaries['team'].apply(map_team_name)
salaries

,year,team,salary
0,2019,Arizona Diamondbacks,62990590
1,2019,Atlanta Braves,115629465
2,2019,Baltimore Orioles,51471496
3,2019,Boston Red Sox,163861035
4,2019,Chicago Cubs,196000476
...,...,...,...
85,2021,St. Louis Cardinals,124483224
86,2021,Tampa Bay Rays,42733384
87,2021,Texas Rangers,44930530
88,2021,Toronto Blue Jays,104271393


In [30]:
teams = get_meta_data('teams')
teams = teams.drop(columns=['teamName'])
teams = teams.rename(columns={'name': 'teamName'})
teams

,id,teamName,teamCode,shortName,abbreviation,locationName,leagueId,leagueName,divisionId,divisionName,venueId,venueName
0,108,Los Angeles Angels,ana,LA Angels,LAA,Anaheim,103,American League,200,American League West,1,Angel Stadium
1,109,Arizona Diamondbacks,ari,Arizona,ARI,Phoenix,104,National League,203,National League West,15,Chase Field
2,110,Baltimore Orioles,bal,Baltimore,BAL,Baltimore,103,American League,201,American League East,2,Oriole Park at Camden Yards
3,111,Boston Red Sox,bos,Boston,BOS,Boston,103,American League,201,American League East,3,Fenway Park
4,112,Chicago Cubs,chn,Chi Cubs,CHC,Chicago,104,National League,205,National League Central,17,Wrigley Field
5,113,Cincinnati Reds,cin,Cincinnati,CIN,Cincinnati,104,National League,205,National League Central,2602,Great American Ball Park
6,114,Cleveland Indians,cle,Cleveland,CLE,Cleveland,103,American League,202,American League Central,5,Progressive Field
7,115,Colorado Rockies,col,Colorado,COL,Denver,104,National League,203,National League West,19,Coors Field
8,116,Detroit Tigers,det,Detroit,DET,Detroit,103,American League,202,American League Central,2394,Comerica Park
9,117,Houston Astros,hou,Houston,HOU,Houston,103,American League,200,American League West,2392,Minute Maid Park


In [31]:
salaries = salaries.merge(teams, left_on='team', right_on='teamName', how='inner')[['id', 'year', 'team', 'salary']].rename(columns={'id': 'teamId', 'team': 'teamName', 'salary': 'teamSalary'})
salaries

,teamId,year,teamName,teamSalary
0,109,2019,Arizona Diamondbacks,62990590
1,109,2020,Arizona Diamondbacks,20167910
2,109,2021,Arizona Diamondbacks,52197674
3,144,2019,Atlanta Braves,115629465
4,144,2020,Atlanta Braves,50392877
...,...,...,...,...
85,141,2020,Toronto Blue Jays,33401637
86,141,2021,Toronto Blue Jays,104271393
87,120,2019,Washington Nationals,153637198
88,120,2020,Washington Nationals,45615023


In [32]:
salaries = salaries.drop_duplicates()
salaries

,teamId,year,teamName,teamSalary
0,109,2019,Arizona Diamondbacks,62990590
1,109,2020,Arizona Diamondbacks,20167910
2,109,2021,Arizona Diamondbacks,52197674
3,144,2019,Atlanta Braves,115629465
4,144,2020,Atlanta Braves,50392877
...,...,...,...,...
85,141,2020,Toronto Blue Jays,33401637
86,141,2021,Toronto Blue Jays,104271393
87,120,2019,Washington Nationals,153637198
88,120,2020,Washington Nationals,45615023


In [33]:
salaries.to_pickle(f'{Config.data_dir}/aggregate/team_salaries.pickle')